# Using Ollama to get structured output

In [1]:
#load "load.fsx"

open Informedica.OpenAI.Lib
open Ollama.Operators

## Define a schema and a type for the output

The json function will output the type used as a type parameter. 

In [8]:

"""
Use schema: { number: int; unit: string }
What is the minimal corrected gestational age mentioned in the text between '''

'''A neonate 28 weeks to 32 weeks corrected gestational age.'''

Reply in JSON."""
|> Message.user
|> Ollama.json<{| number: int; unit: string |}>
    Ollama.Models.llama2
    []
|> Async.RunSynchronously
|> function
    | Ok x -> printfn $"{x}"
    | Error e -> printfn $"oops: {e}"


ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nUse schema: { number: int; unit: string }\nWhat is the minimal corrected gestational age mentioned in the text between '''\n\n'''A neonate 28 weeks to 32 weeks corrected gestational age.'''\n\nReply in JSON.","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty":null,"mirostat":0,"mirostat_tau":null,"mirostat_eta":null,"penalize_newline":null,"stop":[],"numa":null,"num_ctx":2048,"num_batch":null,"num_gqa":null,"num_gpu":null,"main_gpu":null,"low_vram":null,"f16_kv":null,"vocab_only":null,"use_mmap":null,"use_mlock":null,"rope_frequency_base":null,"rope_frequency_scale":null,"num_thread":null},"response_format":{"schema":{
  "$schema": "http://json-schema.org/draft-04/schema#",
  "title": "<>f_

## Getting  the maximum age as a json structure

In [9]:
"""
Use schema: { number: int; unit: string }
What is the maximum corrected gestational age mentioned in the text between '''

'''A neonate 28 weeks to 32 weeks corrected gestational age.'''

Reply in JSON."""
|> Message.user
|> Ollama.json<{| number: int; unit: string |}>
    Ollama.Models.llama2
    []
|> Async.RunSynchronously
|> function
    | Ok x -> printfn $"{x}"
    | Error e -> printfn $"oops: {e}"


ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nUse schema: { number: int; unit: string }\nWhat is the maximum corrected gestational age mentioned in the text between '''\n\n'''A neonate 28 weeks to 32 weeks corrected gestational age.'''\n\nReply in JSON.","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty":null,"mirostat":0,"mirostat_tau":null,"mirostat_eta":null,"penalize_newline":null,"stop":[],"numa":null,"num_ctx":2048,"num_batch":null,"num_gqa":null,"num_gpu":null,"main_gpu":null,"low_vram":null,"f16_kv":null,"vocab_only":null,"use_mmap":null,"use_mlock":null,"rope_frequency_base":null,"rope_frequency_scale":null,"num_thread":null},"response_format":{"schema":{
  "$schema": "http://json-schema.org/draft-04/schema#",
  "title": "<>f_

## Try a different structured output

In [10]:
"""
Use schema: { minAge: int; maxAge: int; unit: string }
What is corrected gestational age range mentioned in the text between '''

'''A neonate 28 weeks to 32 weeks corrected gestational age.'''

Reply in JSON."""
|> Message.user
|> Ollama.json<{| minAge: int; maxAge: int; unit: string |}>
    Ollama.Models.llama2
    []
|> Async.RunSynchronously
|> function
    | Ok x -> printfn $"{x}"
    | Error e -> printfn $"oops: {e}"


ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nUse schema: { minAge: int; maxAge: int; unit: string }\nWhat is corrected gestational age range mentioned in the text between '''\n\n'''A neonate 28 weeks to 32 weeks corrected gestational age.'''\n\nReply in JSON.","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty":null,"mirostat":0,"mirostat_tau":null,"mirostat_eta":null,"penalize_newline":null,"stop":[],"numa":null,"num_ctx":2048,"num_batch":null,"num_gqa":null,"num_gpu":null,"main_gpu":null,"low_vram":null,"f16_kv":null,"vocab_only":null,"use_mmap":null,"use_mlock":null,"rope_frequency_base":null,"rope_frequency_scale":null,"num_thread":null},"response_format":{"schema":{
  "$schema": "http://json-schema.org/draft-04/schema#",
  "title"